# Crypto Backtesting with CCXT Data Adapter

This notebook demonstrates how to use the CCXTAdapter to fetch cryptocurrency data from multiple exchanges and perform a simple backtesting strategy.

## Features Demonstrated:
- Fetching crypto OHLCV data from Binance, Coinbase, and Kraken
- Data validation and schema verification
- Multi-exchange price comparison
- Simple moving average crossover strategy
- Performance metrics calculation

In [ ]:
# Import required libraries
import asyncio
from decimal import Decimal
import pandas as pd
import polars as pl
import matplotlib.pyplot as plt

from rustybt.data.adapters import CCXTAdapter

## 1. Initialize CCXTAdapter

Create adapters for different exchanges. By default, CCXT uses spot markets.

In [ ]:
# Initialize adapters for different exchanges
binance_adapter = CCXTAdapter(exchange_id='binance')
coinbase_adapter = CCXTAdapter(exchange_id='coinbase')
kraken_adapter = CCXTAdapter(exchange_id='kraken')

print(f"Binance adapter initialized: {binance_adapter.name}")
print(f"Coinbase adapter initialized: {coinbase_adapter.name}")
print(f"Kraken adapter initialized: {kraken_adapter.name}")

## 2. Fetch BTC/USDT Data from Binance

Fetch 30 days of hourly BTC/USDT data from Binance.

In [ ]:
# Fetch BTC/USDT data from Binance
btc_data = await binance_adapter.fetch(
    symbols=['BTC/USDT'],
    start_date=pd.Timestamp('2024-01-01'),
    end_date=pd.Timestamp('2024-01-31'),
    resolution='1h'
)

print(f"Fetched {len(btc_data)} rows")
print(f"\nData schema:")
print(btc_data.schema)
print(f"\nFirst 5 rows:")
print(btc_data.head())

## 3. Validate Data Quality

Verify OHLCV relationships and data integrity.

In [ ]:
# Validate data
try:
    binance_adapter.validate(btc_data)
    print("✅ Data validation passed!")
    print(f"   - All OHLCV relationships valid")
    print(f"   - No NULL values")
    print(f"   - Timestamps sorted")
    print(f"   - No duplicate timestamps")
except Exception as e:
    print(f"❌ Validation failed: {e}")

## 4. Multi-Exchange Price Comparison

Compare BTC prices across Binance, Coinbase, and Kraken.

In [ ]:
# Fetch daily BTC data from multiple exchanges
exchanges_data = {}

# Binance: BTC/USDT
binance_btc = await binance_adapter.fetch(
    symbols=['BTC/USDT'],
    start_date=pd.Timestamp('2024-01-01'),
    end_date=pd.Timestamp('2024-01-31'),
    resolution='1d'
)
exchanges_data['Binance'] = binance_btc

# Coinbase: BTC/USD
coinbase_btc = await coinbase_adapter.fetch(
    symbols=['BTC/USD'],
    start_date=pd.Timestamp('2024-01-01'),
    end_date=pd.Timestamp('2024-01-31'),
    resolution='1d'
)
exchanges_data['Coinbase'] = coinbase_btc

# Kraken: BTC/USD
kraken_btc = await kraken_adapter.fetch(
    symbols=['BTC/USD'],
    start_date=pd.Timestamp('2024-01-01'),
    end_date=pd.Timestamp('2024-01-31'),
    resolution='1d'
)
exchanges_data['Kraken'] = kraken_btc

print("Data fetched from all exchanges:")
for exchange, data in exchanges_data.items():
    print(f"  {exchange}: {len(data)} bars")

In [ ]:
# Plot price comparison
plt.figure(figsize=(14, 7))

for exchange, data in exchanges_data.items():
    # Convert to pandas for plotting
    df_pd = data.to_pandas()
    df_pd['close_float'] = df_pd['close'].astype(float)
    
    plt.plot(df_pd['timestamp'], df_pd['close_float'], label=exchange, linewidth=2)

plt.title('BTC Price Comparison Across Exchanges', fontsize=16, fontweight='bold')
plt.xlabel('Date', fontsize=12)
plt.ylabel('Price (USD)', fontsize=12)
plt.legend(fontsize=10)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 5. Simple Moving Average Crossover Strategy

Implement a basic SMA crossover strategy:
- Buy when short SMA crosses above long SMA
- Sell when short SMA crosses below long SMA

In [ ]:
# Calculate moving averages
btc_strategy = btc_data.clone()

# Convert Decimal to float for rolling calculations
btc_strategy = btc_strategy.with_columns([
    pl.col('close').cast(pl.Float64).alias('close_float')
])

# Calculate SMAs
btc_strategy = btc_strategy.with_columns([
    pl.col('close_float').rolling_mean(window_size=20).alias('sma_20'),
    pl.col('close_float').rolling_mean(window_size=50).alias('sma_50'),
])

# Generate signals
btc_strategy = btc_strategy.with_columns([
    (pl.col('sma_20') > pl.col('sma_50')).cast(pl.Int32).alias('signal')
])

# Detect crossovers (signal changes)
btc_strategy = btc_strategy.with_columns([
    (pl.col('signal') - pl.col('signal').shift(1)).alias('position_change')
])

print(f"Strategy data prepared: {len(btc_strategy)} rows")
print(f"\nSignal distribution:")
print(btc_strategy.group_by('signal').agg(pl.count()).sort('signal'))

In [ ]:
# Plot strategy signals
strategy_pd = btc_strategy.to_pandas()

plt.figure(figsize=(14, 7))

# Plot price and SMAs
plt.plot(strategy_pd['timestamp'], strategy_pd['close_float'], 
         label='BTC/USDT Price', linewidth=2, alpha=0.7)
plt.plot(strategy_pd['timestamp'], strategy_pd['sma_20'], 
         label='SMA 20', linewidth=1.5, linestyle='--')
plt.plot(strategy_pd['timestamp'], strategy_pd['sma_50'], 
         label='SMA 50', linewidth=1.5, linestyle='--')

# Mark buy signals (crossover up)
buys = strategy_pd[strategy_pd['position_change'] == 1]
plt.scatter(buys['timestamp'], buys['close_float'], 
           color='green', marker='^', s=200, label='Buy Signal', zorder=5)

# Mark sell signals (crossover down)
sells = strategy_pd[strategy_pd['position_change'] == -1]
plt.scatter(sells['timestamp'], sells['close_float'], 
           color='red', marker='v', s=200, label='Sell Signal', zorder=5)

plt.title('Moving Average Crossover Strategy - BTC/USDT', fontsize=16, fontweight='bold')
plt.xlabel('Date', fontsize=12)
plt.ylabel('Price (USD)', fontsize=12)
plt.legend(fontsize=10)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print(f"\nTotal buy signals: {len(buys)}")
print(f"Total sell signals: {len(sells)}")

## 6. Calculate Simple Performance Metrics

Calculate basic performance metrics for the strategy.

In [ ]:
# Calculate returns
btc_strategy = btc_strategy.with_columns([
    (pl.col('close_float') / pl.col('close_float').shift(1) - 1).alias('returns')
])

# Calculate strategy returns (only when signal = 1)
btc_strategy = btc_strategy.with_columns([
    (pl.col('returns') * pl.col('signal').shift(1)).alias('strategy_returns')
])

# Remove NaN values
btc_strategy_clean = btc_strategy.drop_nulls()

# Calculate cumulative returns
returns_pd = btc_strategy_clean.to_pandas()
returns_pd['cumulative_returns'] = (1 + returns_pd['returns']).cumprod() - 1
returns_pd['cumulative_strategy_returns'] = (1 + returns_pd['strategy_returns']).cumprod() - 1

# Calculate metrics
total_return = returns_pd['cumulative_returns'].iloc[-1]
strategy_return = returns_pd['cumulative_strategy_returns'].iloc[-1]

print(f"\n{'='*50}")
print(f"Performance Metrics (Jan 2024)")
print(f"{'='*50}")
print(f"Buy & Hold Return:    {total_return*100:>8.2f}%")
print(f"Strategy Return:      {strategy_return*100:>8.2f}%")
print(f"Outperformance:       {(strategy_return - total_return)*100:>8.2f}%")
print(f"{'='*50}")

In [ ]:
# Plot cumulative returns
plt.figure(figsize=(14, 7))

plt.plot(returns_pd['timestamp'], returns_pd['cumulative_returns'] * 100, 
         label='Buy & Hold', linewidth=2)
plt.plot(returns_pd['timestamp'], returns_pd['cumulative_strategy_returns'] * 100, 
         label='SMA Crossover Strategy', linewidth=2)

plt.title('Cumulative Returns Comparison', fontsize=16, fontweight='bold')
plt.xlabel('Date', fontsize=12)
plt.ylabel('Cumulative Return (%)', fontsize=12)
plt.legend(fontsize=10)
plt.grid(True, alpha=0.3)
plt.axhline(y=0, color='black', linestyle='-', linewidth=0.5)
plt.tight_layout()
plt.show()

## 7. CCXT Adapter Configuration Options

The CCXTAdapter supports various configuration options:

In [ ]:
# Example: Using testnet mode (if available)
testnet_adapter = CCXTAdapter(
    exchange_id='binance',
    testnet=True
)
print(f"Testnet adapter: {testnet_adapter.name}")

# Example: With API credentials (for private endpoints)
# authenticated_adapter = CCXTAdapter(
#     exchange_id='binance',
#     api_key='your_api_key',
#     api_secret='your_api_secret'
# )

# Supported resolutions
print(f"\nSupported resolutions: {list(binance_adapter.RESOLUTION_MAPPING.keys())}")

# Rate limiting is automatically handled
print(f"\nRate limiter stats:")
print(f"  Requests made: {binance_adapter.rate_limiter.requests_made}")
print(f"  Throttle events: {binance_adapter.rate_limiter.throttle_events}")

## Summary

This notebook demonstrated:

1. ✅ **Data Fetching**: Retrieved crypto OHLCV data from multiple exchanges
2. ✅ **Data Validation**: Verified data quality and OHLCV relationships
3. ✅ **Multi-Exchange Comparison**: Compared prices across Binance, Coinbase, and Kraken
4. ✅ **Strategy Implementation**: Built a simple moving average crossover strategy
5. ✅ **Performance Analysis**: Calculated and visualized strategy returns

### Next Steps

- Implement more sophisticated strategies (RSI, MACD, Bollinger Bands)
- Add transaction costs and slippage
- Use RustyBT's full backtesting engine for complete portfolio simulation
- Optimize strategy parameters using walk-forward analysis
- Test on multiple cryptocurrencies and timeframes